In [1]:
import os
import sys

try:
    import cuda_backend
except:
    from cuda_backend import add
    print('Need to fix import')

Need to fix import


In [2]:
import numpy as np
a = [1, 2, 3]
b = [4, 5, 6]

add(a, b)

array([5, 7, 9], dtype=int32)

In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from utils.matrix import project_points, project_covariance, to_homogeneous
from utils.colmap_data import COLMAP_Data
from utils.plot_utils import create_ellipsoids_as_one_mesh
from src.gaussian import GaussianSplatting
from matplotlib.collections import EllipseCollection
from scipy.stats import chi2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
colmap_data = COLMAP_Data("data/sparse/0")
model = GaussianSplatting(colmap_data)
sigmas = model.init_gaussian_covariance()

E = colmap_data.E[colmap_data.frame_names[1]]
K = colmap_data.K
distortion = colmap_data.distortion
points = colmap_data.points
colors = colmap_data.colors
width = colmap_data.width
height = colmap_data.height
print(colmap_data.frame_names[1])
print(points.shape)

frame_00001.png
(56042, 3)


In [3]:
sigma_2d = project_covariance(points, sigmas, E, K)
points_2d, depths = project_points(points, E, K, return_depth=True)

In [4]:

# naive culling
# bound_indices = (points_2d >= 0).all(axis=1) & \
#     (points_2d < [colmap_data.width, colmap_data.height]).all(axis=1)

## 99% confidence interval in view frustum
"""
99% confidence interval for 2D Gaussian: χ2,2_0.992 ≈3.04
Confidence ellipsoid is scaled by 3.04 times the eigenvalues of the covariance matrix.
"""

def get_covariance_bounds(sigma_2d):
    chi2_threshold = chi2.ppf(0.99, df=2)  # 99% confidence interval for 2D Gaussian

    scale_squared, rotation = np.linalg.eigh(sigma_2d * 0.1)
    extents = np.sqrt(scale_squared * chi2_threshold)
    return extents

bound_indices = []
extents = get_covariance_bounds(sigma_2d)
points_min = points_2d - extents
points_max = points_2d + extents

# Check all bounds conditions in a vectorized way
in_bounds = (points_min[:, 0] > 0) & (points_min[:, 1] > 0) & \
            (points_max[:, 0] < colmap_data.width) & (points_max[:, 1] < colmap_data.height)
bound_indices = np.where(in_bounds)[0]

In [5]:
bounded_points_2d = points_2d[bound_indices]
bounded_colors = colmap_data.colors[bound_indices]
bounded_depths = depths[bound_indices]

bounded_sigmas_2d = sigma_2d[bound_indices]
bounded_sigmas = sigmas[bound_indices]
bounded_extents = extents[bound_indices]

In [6]:
def get_splats_per_tile(points_2d, extents, n=16):
        n = 16
        padding_x = (width % n) // 2
        padding_y = (height % n) // 2

        tiles = np.array(np.meshgrid(np.arange(padding_x, width+padding_x, n), 
                                np.arange(padding_y, height+padding_y, n))).T.reshape(-1, 2)

        tiles_expanded = tiles[:, np.newaxis, :]  # Shape: (2135, 1, 2)
        points_expanded = points_2d[np.newaxis, :, :]  # Shape: (1, 8108, 2)
        extent_expanded = extents[np.newaxis, :, :]  # Shape: (1, 8108, 2)

        x_dist = (tiles_expanded[:, :, 0] - points_expanded[:, :, 0])**2  # Shape: (2135, 8108)
        y_dist = (tiles_expanded[:, :, 1] - points_expanded[:, :, 1])**2  # Shape: (2135, 8108)

        dist = (x_dist / extent_expanded[:, :, 0]**2 + y_dist / extent_expanded[:, :, 1]**2)  # Shape: (2135, 8108)

        in_ellipse_mask = dist <= 1  # Shape: (2135, 8108)

        points_indices_per_tile = [np.where(in_ellipse_mask[i])[0] for i in range(len(tiles))]
        return points_indices_per_tile

In [7]:
# for each tile, we have a list of gaussian indices

# convert them to [depth - tile_idx] 64 bits and radix sort

depth_bits = bounded_depths.astype(np.float32)

In [8]:
points_indices_per_tile = get_splats_per_tile(bounded_points_2d, bounded_extents)